# Numero de Nusselt con PySR

In [1]:
## Importando Librerias 
import numpy as np
import pandas as pd
from pysr import PySRRegressor
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


### Cargando datos

In [2]:
# importacion de DF
file_path = 'C:\\Users\\lucas\\Documents\\VSCode\\CODES\\LIB_SR\\Dataset_coeficientes\\df_n_25.txt'
df_n_25 = pd.read_csv(file_path, delimiter=',')

file_path = 'C:\\Users\\lucas\\Documents\\VSCode\\CODES\\LIB_SR\\Dataset_coeficientes\\df_n_53.txt'
df_n_53 = pd.read_csv(file_path, delimiter=',')

file_path = 'C:\\Users\\lucas\\Documents\\VSCode\\CODES\\LIB_SR\\Dataset_coeficientes\\df_n_74.txt'
df_n_74 = pd.read_csv(file_path, delimiter=',')

file_path = 'C:\\Users\\lucas\\Documents\\VSCode\\CODES\\LIB_SR\\Dataset_coeficientes\\df_n_102.txt'
df_n_102 = pd.read_csv(file_path, delimiter=',')

In [3]:
# definiendo conjuntos de train y test

# definiendo conjunto de train
df_n_train = pd.concat([df_n_25, df_n_74], ignore_index=True)

# separando entre x e y
y_train = df_n_train.drop(columns=['Current','K','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','Rem','prandtl','colIndex'])
X_train = df_n_train.drop(columns=['Current','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','colIndex','nusselt'])


# definiendo conjunto de test

y_test = df_n_53.drop(columns=['Current','K','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','Rem','prandtl','colIndex'])
X_test = df_n_53.drop(columns=['Current','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','colIndex','nusselt'])

# renombrando
X_train.rename(columns={'Rem':'Re'}, inplace=True)
X_test.rename(columns={'Rem':'Re'}, inplace=True)

### Entrenamiento

In [17]:
model = PySRRegressor(
    niterations=100,
    binary_operators=["*", "pow"],
    unary_operators=[],
    population_size=200,
    verbosity=1,
    maxsize=12,
    parsimony=1e-3,
    progress=True
)

model.fit(X_train, y_train)

d:\env\lib\site-packages\pysr\sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
d:\env\lib\site-packages\pysr\sr.py:2029: UserWarning: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (https://astroautomata.com/PySR/options/#batching). You should also reconsider if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form with symbolic regression. More datapoints will lower the search speed.
  warnings.warn(
d:\env\lib\site-packages\pysr\sr.py:73: UserWarning: You are using the `^` operator, but have not set up `constraints` for it. This may lead to overly complex expressions. One typical constraint is to use `constraints={..., '^': (-1, 1)}`, which will allow arbitrary-complexity base (-1) but only powers such as a constant or varia

Compiling Julia backend...


[ Info: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (`options.batching`), and also if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form.
[ Info: Started!



Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 1500 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 1500 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 3.590e+01
Head worker occupation: 0.0%
Progress: 1 / 1500 total iterations (0.067%)
Hall of Fame:
-------------

PySRRegressor.equations_ = [
	   pick     score                                           equation  \
	0        0.000000                                           58.26644   
	1        1.272364                                     Re ^ 0.5084616   
	2        0.251771                        (Re ^ 0.6168856) * 0.404387   
	3        0.091353                (Re ^ 0.5367839) * (0.78962463 ^ K)   
	4  >>>>  0.672898  (K ^ -0.19482446) * ((Re * 0.18293631) ^ 0.636...   
	5        0.019082  (K ^ -0.19693369) * ((0.18614472 * Re) ^ (0.52...   
	
	         loss  complexity  
	0  477.439880           1  
	1   37.476387           3  
	2   22.650220           5  
	3   18.867940           7  
	4    4.911934           9  
	5    4.728008          11  
]

In [22]:
best_equation = model.equations_.iloc[2]['equation']
print(best_equation)

(Re ^ 0.6168856) * 0.404387


In [21]:
best_equation = model.equations_.iloc[3]['equation']
print(best_equation)

(Re ^ 0.5367839) * (0.78962463 ^ K)


In [19]:
best_equation = model.equations_.iloc[4]['equation']
print(best_equation)

(K ^ -0.19482446) * ((Re * 0.18293631) ^ 0.6367587)


In [20]:
best_equation = model.equations_.iloc[5]['equation']
print(best_equation)

(K ^ -0.19693369) * ((0.18614472 * Re) ^ (0.5267377 ^ prandtl))


In [10]:
best_equation = model.equations_.iloc[3]['equation']
print(best_equation)

(Re ^ 0.5300617) + (-13.170971 * K)


In [14]:
X_test

,K,Re,prandtl
0,0.6,2824.604561,0.709843
1,0.6,2824.604561,0.709843
2,0.6,2352.302465,0.709842
3,0.6,2000.632597,0.709842
4,0.6,1878.735604,0.709842
...,...,...,...
8605,1.2,9267.885175,0.707768
8606,1.2,8959.387980,0.707705
8607,1.2,8691.558442,0.707625
8608,1.2,8537.950633,0.707554


### Pruebas

In [23]:
# Predecir en los datos de prueba
y_pred = model.predict(X_test)

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("R^2:", r2)

MSE: 4.860948813078121
R^2: 0.9899781969290627


In [6]:
# con la mejor ecuacion

def nusselt(S, Re, Pr):
    return 0.5 * S **(-0.2) * Re **(0.64) * Pr

def nusselt_gp(S, Re, Pr):
    return (S ** (-0.19482446)) * ((Re * 0.18293631) ** 0.6367587)

In [7]:
# Calcular cdrag_pred y cdrag_gp_pred en vectores separados
cdrag_pred = X_test.apply(lambda row: nusselt(row['K'], row['Re'], row['prandtl']), axis=1).values
cdrag_gp_pred = X_test.apply(lambda row: nusselt_gp(row['K'], row['Re'], row['prandtl']), axis=1).values

# Calcular R^2 y MSE para cdrag
r2_cdrag = r2_score(y_test['nusselt'], cdrag_pred)
mse_cdrag = mean_squared_error(y_test['nusselt'], cdrag_pred)

# Calcular MAE y MAPE para cdrag
mae_cdrag = mean_absolute_error(y_test['nusselt'], cdrag_pred)
mape_cdrag = mean_absolute_percentage_error(y_test['nusselt'], cdrag_pred)

# Calcular R^2 y MSE para cdrag_gp
r2_cdrag_gp = r2_score(y_test['nusselt'], cdrag_gp_pred)
mse_cdrag_gp = mean_squared_error(y_test['nusselt'], cdrag_gp_pred)

# Calcular MAE y MAPE para cdrag_gp
mae_cdrag_gp = mean_absolute_error(y_test['nusselt'], cdrag_gp_pred)
mape_cdrag_gp = mean_absolute_percentage_error(y_test['nusselt'], cdrag_gp_pred)

#  Mostrar Metricas 

print(f"R^2 cdrag original: {r2_cdrag}, R^2 cdrag GPLearn: {r2_cdrag_gp} ")
print(f"MSE cdrag original: {mse_cdrag}, MSE cdrag GPLearn: {mse_cdrag_gp} ")
print(f"MAE cdrag original: {mae_cdrag}, MAE cdrag GPLearn: {mae_cdrag_gp} ")
print(f"MAPE cdrag original: {mape_cdrag*100}*100, MAPE cdrag GPLearn: {mape_cdrag_gp*100}*100 ")

R^2 cdrag original: 0.9509992536047758, R^2 cdrag GPLearn: 0.9899781969290627 
MSE cdrag original: 23.767192225174124, MSE cdrag GPLearn: 4.860948813078121 
MAE cdrag original: 4.174721418276441, MAE cdrag GPLearn: 1.58687980229672 
MAPE cdrag original: 7.197154998836116*100, MAPE cdrag GPLearn: 2.770234544358197*100 


In [8]:
# con ecuacion con sentido físico 

def nusselt(S, Re, Pr):
    return 0.5 * S **(-0.2) * Re **(0.64) * Pr

def nusselt_gp(S, Re, Pr):
    return (S **( -0.19693369)) * ((0.18614472 * Re) ** (0.5267377 ** Pr))

In [9]:
# Calcular cdrag_pred y cdrag_gp_pred en vectores separados
cdrag_pred = X_test.apply(lambda row: nusselt(row['K'], row['Re'], row['prandtl']), axis=1).values
cdrag_gp_pred = X_test.apply(lambda row: nusselt_gp(row['K'], row['Re'], row['prandtl']), axis=1).values

# Calcular R^2 y MSE para cdrag
r2_cdrag = r2_score(y_test['nusselt'], cdrag_pred)
mse_cdrag = mean_squared_error(y_test['nusselt'], cdrag_pred)

# Calcular MAE y MAPE para cdrag
mae_cdrag = mean_absolute_error(y_test['nusselt'], cdrag_pred)
mape_cdrag = mean_absolute_percentage_error(y_test['nusselt'], cdrag_pred)

# Calcular R^2 y MSE para cdrag_gp
r2_cdrag_gp = r2_score(y_test['nusselt'], cdrag_gp_pred)
mse_cdrag_gp = mean_squared_error(y_test['nusselt'], cdrag_gp_pred)

# Calcular MAE y MAPE para cdrag_gp
mae_cdrag_gp = mean_absolute_error(y_test['nusselt'], cdrag_gp_pred)
mape_cdrag_gp = mean_absolute_percentage_error(y_test['nusselt'], cdrag_gp_pred)

#  Mostrar Metricas 

print(f"R^2 cdrag original: {r2_cdrag}, R^2 cdrag GPLearn: {r2_cdrag_gp} ")
print(f"MSE cdrag original: {mse_cdrag}, MSE cdrag GPLearn: {mse_cdrag_gp} ")
print(f"MAE cdrag original: {mae_cdrag}, MAE cdrag GPLearn: {mae_cdrag_gp} ")
print(f"MAPE cdrag original: {mape_cdrag*100}*100, MAPE cdrag GPLearn: {mape_cdrag_gp*100}*100 ")

R^2 cdrag original: 0.9509992536047758, R^2 cdrag GPLearn: 0.9901753880138298 
MSE cdrag original: 23.767192225174124, MSE cdrag GPLearn: 4.765303771695522 
MAE cdrag original: 4.174721418276441, MAE cdrag GPLearn: 1.58082670555206 
MAPE cdrag original: 7.197154998836116*100, MAPE cdrag GPLearn: 2.7589904051335967*100 


### Experimento V2

In [4]:
model = PySRRegressor(
    niterations=50,
    binary_operators=["+","-","*" ,"/", "^"],
    unary_operators=[],
    population_size=200,
    verbosity=1,
    maxsize=15,
    parsimony=1e-3,
    constraints={'^':(2,1)},
    nested_constraints={'^':{'^':0}},
    complexity_of_variables = 2,
    random_state = 0,
    progress=True
)

model.fit(X_train, y_train)

d:\env\lib\site-packages\pysr\sr.py:1310: UserWarning: Note: Setting `random_state` without also setting `deterministic` to True and `procs` to 0 will result in non-deterministic searches. 
  warnings.warn(
d:\env\lib\site-packages\pysr\sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
d:\env\lib\site-packages\pysr\sr.py:2029: UserWarning: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (https://astroautomata.com/PySR/options/#batching). You should also reconsider if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form with symbolic regression. More datapoints will lower the search speed.
  warnings.warn(


Compiling Julia backend...


[ Info: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (`options.batching`), and also if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form.
[ Info: Started!



Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 750 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 2.560e+02
Head worker occupation: 0.0%
Progress: 1 / 750 total iterations (0.133%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
3           3.702e+03  5.314e+00  y = 0.71841 + 0.76102
4           4.774e+02  2.048e+00  y = prandtl + 57.561
10          4.767e+02  2.457e-04  y = (((111.01 / -1072.6) + -24.781) / prandtl) + 93.406
15          5.525e+01  4.310e-01  y = ((((0.34641 + Re) / 185.64) + (16.562 / 0.80218)) / prandt...

PySRRegressor.equations_ = [
	   pick     score                                           equation  \
	0        0.000000                                          29.486423   
	1        0.503091                              30.623444 + 27.646608   
	2        0.977268                                   Re * 0.015386766   
	3        1.035482                      (Re ^ 0.61681247) / 2.4725173   
	4        0.004474        ((Re ^ 0.69425625) / 5.299438) - -7.1390924   
	5  >>>>  1.145420                (K * -13.156487) + (Re ^ 0.5300428)   
	6        0.116432   ((Re ^ 0.6714729) / (3.1073892 + K)) + 3.1073892   
	7        0.052250  ((Re ^ 0.6282893) / (3.104769 + (-0.3583564 / ...   
	8        0.045426  ((Re ^ 0.62598604) / ((-0.32713947 / K) + 3.00...   
	9        0.002420  ((Re ^ 0.60988355) / ((-0.5727791 / (K + 0.246...   
	
	          loss  complexity  
	0  1305.864600           1  
	1   477.439880           3  
	2   179.678710           4  
	3    22.651072           6  
	4    22.449290           8  
	5     7.140899           9  
	6     5.657455          11  
	7     5.096091          13  
	8     4.869773          14  
	9     4.858000          15  
]

In [5]:
best_equation = model.equations_.iloc[5]['equation']
print(best_equation)

(K * -13.156487) + (Re ^ 0.5300428)


In [6]:
best_equation = model.equations_.iloc[6]['equation']
print(best_equation)

((Re ^ 0.6714729) / (3.1073892 + K)) + 3.1073892


In [7]:
best_equation = model.equations_.iloc[7]['equation']
print(best_equation)

((Re ^ 0.6282893) / (3.104769 + (-0.3583564 / K))) + -0.61090964


In [8]:
best_equation = model.equations_.iloc[8]['equation']
print(best_equation)

((Re ^ 0.62598604) / ((-0.32713947 / K) + 3.0008318)) - K


In [9]:
best_equation = model.equations_.iloc[9]['equation']
print(best_equation)

((Re ^ 0.60988355) / ((-0.5727791 / (K + 0.24684739)) + 2.7714062)) + -1.6077693


In [10]:
# mejores ecuaciones

def nusselt(S, Re, Pr):
    return 0.5 * S **(-0.2) * Re **(0.64) * Pr

def nusselt_gp(S, Re, Pr):
    return (S * (-13.156487)) + (Re ** 0.5300428)


In [11]:
# Calcular cdrag_pred y cdrag_gp_pred en vectores separados
cdrag_pred = X_test.apply(lambda row: nusselt(row['K'], row['Re'], row['prandtl']), axis=1).values
cdrag_gp_pred = X_test.apply(lambda row: nusselt_gp(row['K'], row['Re'], row['prandtl']), axis=1).values

# Calcular R^2 y MSE para cdrag
r2_cdrag = r2_score(y_test['nusselt'], cdrag_pred)
mse_cdrag = mean_squared_error(y_test['nusselt'], cdrag_pred)

# Calcular MAE y MAPE para cdrag
mae_cdrag = mean_absolute_error(y_test['nusselt'], cdrag_pred)
mape_cdrag = mean_absolute_percentage_error(y_test['nusselt'], cdrag_pred)

# Calcular R^2 y MSE para cdrag_gp
r2_cdrag_gp = r2_score(y_test['nusselt'], cdrag_gp_pred)
mse_cdrag_gp = mean_squared_error(y_test['nusselt'], cdrag_gp_pred)

# Calcular MAE y MAPE para cdrag_gp
mae_cdrag_gp = mean_absolute_error(y_test['nusselt'], cdrag_gp_pred)
mape_cdrag_gp = mean_absolute_percentage_error(y_test['nusselt'], cdrag_gp_pred)

#  Mostrar Metricas 

print(f"R^2 cdrag original: {r2_cdrag}, R^2 cdrag GPLearn: {r2_cdrag_gp} ")
print(f"MSE cdrag original: {mse_cdrag}, MSE cdrag GPLearn: {mse_cdrag_gp} ")
print(f"MAE cdrag original: {mae_cdrag}, MAE cdrag GPLearn: {mae_cdrag_gp} ")
print(f"MAPE cdrag original: {mape_cdrag*100}*100, MAPE cdrag GPLearn: {mape_cdrag_gp*100}*100 ")

R^2 cdrag original: 0.9509992536047758, R^2 cdrag GPLearn: 0.9839676498782285 
MSE cdrag original: 23.767192225174124, MSE cdrag GPLearn: 7.776288632260021 
MAE cdrag original: 4.174721418276441, MAE cdrag GPLearn: 1.7947292758808802 
MAPE cdrag original: 7.197154998836116*100, MAPE cdrag GPLearn: 3.0704770140844486*100 


In [12]:
# ecuacion con sentido físico

def nusselt(S, Re, Pr):
    return 0.5 * S **(-0.2) * Re **(0.64) * Pr

def nusselt_gp(S, Re, Pr):
    return ((Re ** 0.6714729) / (3.1073892 + S)) + 3.1073892


In [13]:
# Calcular cdrag_pred y cdrag_gp_pred en vectores separados
cdrag_pred = X_test.apply(lambda row: nusselt(row['K'], row['Re'], row['prandtl']), axis=1).values
cdrag_gp_pred = X_test.apply(lambda row: nusselt_gp(row['K'], row['Re'], row['prandtl']), axis=1).values

# Calcular R^2 y MSE para cdrag
r2_cdrag = r2_score(y_test['nusselt'], cdrag_pred)
mse_cdrag = mean_squared_error(y_test['nusselt'], cdrag_pred)

# Calcular MAE y MAPE para cdrag
mae_cdrag = mean_absolute_error(y_test['nusselt'], cdrag_pred)
mape_cdrag = mean_absolute_percentage_error(y_test['nusselt'], cdrag_pred)

# Calcular R^2 y MSE para cdrag_gp
r2_cdrag_gp = r2_score(y_test['nusselt'], cdrag_gp_pred)
mse_cdrag_gp = mean_squared_error(y_test['nusselt'], cdrag_gp_pred)

# Calcular MAE y MAPE para cdrag_gp
mae_cdrag_gp = mean_absolute_error(y_test['nusselt'], cdrag_gp_pred)
mape_cdrag_gp = mean_absolute_percentage_error(y_test['nusselt'], cdrag_gp_pred)

#  Mostrar Metricas 

print(f"R^2 cdrag original: {r2_cdrag}, R^2 cdrag GPLearn: {r2_cdrag_gp} ")
print(f"MSE cdrag original: {mse_cdrag}, MSE cdrag GPLearn: {mse_cdrag_gp} ")
print(f"MAE cdrag original: {mae_cdrag}, MAE cdrag GPLearn: {mae_cdrag_gp} ")
print(f"MAPE cdrag original: {mape_cdrag*100}*100, MAPE cdrag GPLearn: {mape_cdrag_gp*100}*100 ")

R^2 cdrag original: 0.9509992536047758, R^2 cdrag GPLearn: 0.9887093383371796 
MSE cdrag original: 23.767192225174124, MSE cdrag GPLearn: 5.476392623190977 
MAE cdrag original: 4.174721418276441, MAE cdrag GPLearn: 1.6254529639320818 
MAPE cdrag original: 7.197154998836116*100, MAPE cdrag GPLearn: 2.78002206244416*100 


### Experimento: RMSLE como funcion loss

In [5]:


rmsle_loss = """ function rmsle(prediction, target)
    # Calcular los logaritmos con un mínimo de 0.001 para evitar valores negativos o cero
    log_true = log.(max.(target, 0.001) .+ 1)
    log_pred = log.(max.(prediction, 0.001) .+ 1)

    # Calcular el error cuadrático medio logarítmico
    rmsle_value = (log_pred .- log_true).^2

    # Calcular el valor de RMSLE como la raíz cuadrada de la suma entre la cantidad de elementos
    return sqrt(sum(rmsle_value) / length(target))
end
"""

In [6]:
model = PySRRegressor(
    niterations=30,
    binary_operators=["+","-","*" ,"/", "^"],
    unary_operators=[],
    elementwise_loss= rmsle_loss,
    population_size=200,
    verbosity=1,
    maxsize=15,
    parsimony=1e-3,
    constraints={'^':(2,1)},
    nested_constraints={'^':{'^':0}},
    complexity_of_variables = 2,
    random_state = 0,
    progress=True
)

model.fit(X_train, y_train)

c:\Users\lucas\Documents\VSCode\env\Lib\site-packages\pysr\sr.py:1310: UserWarning: Note: Setting `random_state` without also setting `deterministic` to True and `procs` to 0 will result in non-deterministic searches. 
  warnings.warn(
c:\Users\lucas\Documents\VSCode\env\Lib\site-packages\pysr\sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
c:\Users\lucas\Documents\VSCode\env\Lib\site-packages\pysr\sr.py:2029: UserWarning: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (https://astroautomata.com/PySR/options/#batching). You should also reconsider if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form with symbolic regression. More datapoints will lower the search speed.
  warnings.warn(


Compiling Julia backend...


[ Info: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (`options.batching`), and also if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form.
[ Info: Started!



Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 450 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 450 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 450 total iterations (0.000%)
Hall of Fame:
----------------

PySRRegressor.equations_ = [
	   pick     score                                           equation  \
	0        0.000000                                          1.6702882   
	1        1.228603                             23.143505 - -24.531715   
	2        0.441146                                      Re / 50.97014   
	3        0.496532                      (Re ^ 0.62056303) / 2.6253293   
	4        0.006218        10.933006 + ((Re ^ 0.75991166) / 10.149425)   
	5  >>>>  0.738809                 (Re ^ 0.6620367) / (2.5865386 + K)   
	6        0.042586  ((Re ^ 0.66685677) / (2.9065733 + K)) - -2.481...   
	7        0.002014  (((Re ^ 0.66717637) + 3.4073603) / (2.9065733 ...   
	8        0.000190  ((((Re ^ 0.66717637) + 3.4073603) / (2.9065733...   
	
	       loss  complexity  
	0  3.039944           1  
	1  0.260444           3  
	2  0.167543           4  
	3  0.062065           6  
	4  0.061298           8  
	5  0.029281           9  
	6  0.026890          11  
	7  0.026782          13  
	8  0.026772          15  
]

In [7]:
best_equation = model.equations_.iloc[5]['equation']
print(best_equation)

(Re ^ 0.6620367) / (2.5865386 + K)


In [8]:
best_equation = model.equations_.iloc[6]['equation']
print(best_equation)

((Re ^ 0.66685677) / (2.9065733 + K)) - -2.4812217


In [9]:
best_equation = model.equations_.iloc[7]['equation']
print(best_equation)

(((Re ^ 0.66717637) + 3.4073603) / (2.9065733 + K)) - -1.5185823


In [11]:
best_equation = model.equations_.iloc[8]['equation']
print(best_equation)

((((Re ^ 0.66717637) + 3.4073603) / (2.9065733 + K)) - -1.5185823) + -0.03125198


In [14]:
# ecuacion con sentido físico

def nusselt(S, Re, Pr):
    return 0.5 * S **(-0.2) * Re **(0.64) * Pr

def nusselt_gp(S, Re, Pr):
    return (Re ** 0.6620367) / (2.5865386 + S)


In [15]:
# Calcular nusselt_pred y nusselt_gp_pred en vectores separados
nusselt_pred = X_test.apply(lambda row: nusselt(row['K'], row['Re'], row['prandtl']), axis=1).values
nusselt_gp_pred = X_test.apply(lambda row: nusselt_gp(row['K'], row['Re'], row['prandtl']), axis=1).values

# Calcular R^2 y MSE para nusselt
r2_nusselt = r2_score(y_test['nusselt'], nusselt_pred)
mse_nusselt = mean_squared_error(y_test['nusselt'], nusselt_pred)

# Calcular MAE y MAPE para nusselt
mae_nusselt = mean_absolute_error(y_test['nusselt'], nusselt_pred)
mape_nusselt = mean_absolute_percentage_error(y_test['nusselt'], nusselt_pred)

# Calcular R^2 y MSE para nusselt_gp
r2_nusselt_gp = r2_score(y_test['nusselt'], nusselt_gp_pred)
mse_nusselt_gp = mean_squared_error(y_test['nusselt'], nusselt_gp_pred)

# Calcular MAE y MAPE para nusselt_gp
mae_nusselt_gp = mean_absolute_error(y_test['nusselt'], nusselt_gp_pred)
mape_nusselt_gp = mean_absolute_percentage_error(y_test['nusselt'], nusselt_gp_pred)

# Mostrar Métricas
print(f"R^2 nusselt original: {r2_nusselt}, R^2 nusselt GPLearn: {r2_nusselt_gp} ")
print(f"MSE nusselt original: {mse_nusselt}, MSE nusselt GPLearn: {mse_nusselt_gp} ")
print(f"MAE nusselt original: {mae_nusselt}, MAE nusselt GPLearn: {mae_nusselt_gp} ")
print(f"MAPE nusselt original: {mape_nusselt*100} %, MAPE nusselt GPLearn: {mape_nusselt_gp*100} % ")


R^2 nusselt original: 0.9509992536047758, R^2 nusselt GPLearn: 0.9871730009687187 
MSE nusselt original: 23.767192225174124, MSE nusselt GPLearn: 6.2215736305252936 
MAE nusselt original: 4.174721418276441, MAE nusselt GPLearn: 1.662246484729591 
MAPE nusselt original: 7.197154998836116 %, MAPE nusselt GPLearn: 2.8384569560565964 % 


In [5]:
# Ecuacion 7

def nusselt(S, Re, Pr):
    return 0.5 * S **(-0.2) * Re **(0.64) * Pr

def nusselt_gp(S, Re, Pr):
    return (((Re ** 0.66717637) + 3.4073603) / (2.9065733 + S)) - -1.5185823

# Calcular nusselt_pred y nusselt_gp_pred en vectores separados
nusselt_pred = X_test.apply(lambda row: nusselt(row['K'], row['Re'], row['prandtl']), axis=1).values
nusselt_gp_pred = X_test.apply(lambda row: nusselt_gp(row['K'], row['Re'], row['prandtl']), axis=1).values

# Calcular R^2 y MSE para nusselt
r2_nusselt = r2_score(y_test['nusselt'], nusselt_pred)
mse_nusselt = mean_squared_error(y_test['nusselt'], nusselt_pred)

# Calcular MAE y MAPE para nusselt
mae_nusselt = mean_absolute_error(y_test['nusselt'], nusselt_pred)
mape_nusselt = mean_absolute_percentage_error(y_test['nusselt'], nusselt_pred)

# Calcular R^2 y MSE para nusselt_gp
r2_nusselt_gp = r2_score(y_test['nusselt'], nusselt_gp_pred)
mse_nusselt_gp = mean_squared_error(y_test['nusselt'], nusselt_gp_pred)

# Calcular MAE y MAPE para nusselt_gp
mae_nusselt_gp = mean_absolute_error(y_test['nusselt'], nusselt_gp_pred)
mape_nusselt_gp = mean_absolute_percentage_error(y_test['nusselt'], nusselt_gp_pred)

# Mostrar Métricas
print(f"R^2 nusselt original: {r2_nusselt}, R^2 nusselt GPLearn: {r2_nusselt_gp} ")
print(f"MSE nusselt original: {mse_nusselt}, MSE nusselt GPLearn: {mse_nusselt_gp} ")
print(f"MAE nusselt original: {mae_nusselt}, MAE nusselt GPLearn: {mae_nusselt_gp} ")
print(f"MAPE nusselt original: {mape_nusselt*100} %, MAPE nusselt GPLearn: {mape_nusselt_gp*100} % ")

R^2 nusselt original: 0.9509992536047758, R^2 nusselt GPLearn: 0.9890675303450868 
MSE nusselt original: 23.767192225174124, MSE nusselt GPLearn: 5.302656120551034 
MAE nusselt original: 4.174721418276441, MAE nusselt GPLearn: 1.5722634866499428 
MAPE nusselt original: 7.197154998836116 %, MAPE nusselt GPLearn: 2.6818935619896003 % 


In [ ]:
# con otra ecuacion

def nusselt(S, Re, Pr):
    return 0.5 * S **(-0.2) * Re **(0.64) * Pr

def nusselt_gp(S, Re, Pr):
    return (((Re ** 0.66717637) + 3.4073603) / (2.9065733 + S)) + 1.5185823

# Calcular nusselt_pred y nusselt_gp_pred en vectores separados
nusselt_pred = X_test.apply(lambda row: nusselt(row['K'], row['Re'], row['prandtl']), axis=1).values
nusselt_gp_pred = X_test.apply(lambda row: nusselt_gp(row['K'], row['Re'], row['prandtl']), axis=1).values

# Calcular R^2 y MSE para nusselt
r2_nusselt = r2_score(y_test['nusselt'], nusselt_pred)
mse_nusselt = mean_squared_error(y_test['nusselt'], nusselt_pred)

# Calcular MAE y MAPE para nusselt
mae_nusselt = mean_absolute_error(y_test['nusselt'], nusselt_pred)
mape_nusselt = mean_absolute_percentage_error(y_test['nusselt'], nusselt_pred)

# Calcular R^2 y MSE para nusselt_gp
r2_nusselt_gp = r2_score(y_test['nusselt'], nusselt_gp_pred)
mse_nusselt_gp = mean_squared_error(y_test['nusselt'], nusselt_gp_pred)

# Calcular MAE y MAPE para nusselt_gp
mae_nusselt_gp = mean_absolute_error(y_test['nusselt'], nusselt_gp_pred)
mape_nusselt_gp = mean_absolute_percentage_error(y_test['nusselt'], nusselt_gp_pred)

# Mostrar Métricas
print(f"R^2 nusselt original: {r2_nusselt}, R^2 nusselt GPLearn: {r2_nusselt_gp} ")
print(f"MSE nusselt original: {mse_nusselt}, MSE nusselt GPLearn: {mse_nusselt_gp} ")
print(f"MAE nusselt original: {mae_nusselt}, MAE nusselt GPLearn: {mae_nusselt_gp} ")
print(f"MAPE nusselt original: {mape_nusselt*100} %, MAPE nusselt GPLearn: {mape_nusselt_gp*100} % ")

In [ ]:
model = PySRRegressor(
    niterations=30,
    binary_operators=["+","-","*" ,"/", "^"],
    unary_operators=[],
    elementwise_loss= rmsle_loss,
    population_size=200,
    verbosity=1,
    maxsize=15,
    parsimony=1e-3,
    constraints={'^':(2,1)},
    nested_constraints={'^':{'^':0}},
    complexity_of_variables = 2,
    random_state = 0,
    progress=True
)

model.fit(X_train, y_train)

### Aumentando el valor del parsimony

In [4]:
rmsle_loss = """ function rmsle(prediction, target)
    # Calcular los logaritmos con un mínimo de 0.001 para evitar valores negativos o cero
    log_true = log.(max.(target, 0.001) .+ 1)
    log_pred = log.(max.(prediction, 0.001) .+ 1)

    # Calcular el error cuadrático medio logarítmico
    rmsle_value = (log_pred .- log_true).^2

    # Calcular el valor de RMSLE como la raíz cuadrada de la suma entre la cantidad de elementos
    return sqrt(sum(rmsle_value) / length(target))
end
"""

In [5]:
model = PySRRegressor(
    niterations=30,
    binary_operators=["+","-","*" ,"/", "^"],
    unary_operators=[],
    elementwise_loss= rmsle_loss,
    population_size=200,
    verbosity=1,
    maxsize=15,
    parsimony=0.1,
    constraints={'^':(-1,1)},
    #nested_constraints={'^':{'^':0}},
    complexity_of_variables = 2,
    random_state = 0,
    progress=True
)

model.fit(X_train, y_train)

c:\Users\lucas\Documents\VSCode\env\Lib\site-packages\pysr\sr.py:1310: UserWarning: Note: Setting `random_state` without also setting `deterministic` to True and `procs` to 0 will result in non-deterministic searches. 
  warnings.warn(
c:\Users\lucas\Documents\VSCode\env\Lib\site-packages\pysr\sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
c:\Users\lucas\Documents\VSCode\env\Lib\site-packages\pysr\sr.py:2029: UserWarning: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (https://astroautomata.com/PySR/options/#batching). You should also reconsider if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form with symbolic regression. More datapoints will lower the search speed.
  warnings.warn(


Compiling Julia backend...


[ Info: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (`options.batching`), and also if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form.
[ Info: Started!



Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 450 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 450 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 450 total iterations (0.000%)
Hall of Fame:
----------------

PySRRegressor.equations_ = [
	   pick     score                                           equation  \
	0        0.000000                                          2.4968073   
	1        1.182158                              23.425192 + 24.261206   
	2        1.162613                                     Re ^ 0.5055114   
	3        0.135794                      (Re ^ 0.6212799) * 0.37868556   
	4        0.000728        ((0.1974224 * Re) ^ 0.62549454) + 0.5595231   
	5  >>>>  0.732018               (Re ^ 0.5269304) - (K / 0.086386316)   
	6        0.002598   ((Re ^ 0.5267087) + 0.708955) - (K / 0.08209341)   
	7        0.052041  ((Re / ((K - -1.5562664) / 0.30350396)) ^ 0.67...   
	8        0.000577  (((Re / ((K - -1.5562664) / 0.30350396)) ^ 0.6...   
	
	       loss  complexity  
	0  2.770281           1  
	1  0.260444           3  
	2  0.081433           4  
	3  0.062065           6  
	4  0.061975           8  
	5  0.029806           9  
	6  0.029652          11  
	7  0.026721          13  
	8  0.026690          15  
]

In [6]:
best_equation = model.equations_.iloc[5]['equation']
print(best_equation)

(Re ^ 0.5269304) - (K / 0.086386316)


In [7]:
best_equation = model.equations_.iloc[6]['equation']
print(best_equation)

((Re ^ 0.5267087) + 0.708955) - (K / 0.08209341)


In [8]:
best_equation = model.equations_.iloc[7]['equation']
print(best_equation)

((Re / ((K - -1.5562664) / 0.30350396)) ^ 0.6763588) + 2.6371925


In [9]:
best_equation = model.equations_.iloc[8]['equation']
print(best_equation)

(((Re / ((K - -1.5562664) / 0.30350396)) ^ 0.6763588) - 0.09196712) + 2.6371925


In [10]:
# Ecuacion 5

def nusselt(S, Re, Pr):
    return 0.5 * S **(-0.2) * Re **(0.64) * Pr

def nusselt_gp(S, Re, Pr):
    return (Re ** 0.5269304) - (S / 0.086386316)

# Calcular nusselt_pred y nusselt_gp_pred en vectores separados
nusselt_pred = X_test.apply(lambda row: nusselt(row['K'], row['Re'], row['prandtl']), axis=1).values
nusselt_gp_pred = X_test.apply(lambda row: nusselt_gp(row['K'], row['Re'], row['prandtl']), axis=1).values

# Calcular R^2 y MSE para nusselt
r2_nusselt = r2_score(y_test['nusselt'], nusselt_pred)
mse_nusselt = mean_squared_error(y_test['nusselt'], nusselt_pred)

# Calcular MAE y MAPE para nusselt
mae_nusselt = mean_absolute_error(y_test['nusselt'], nusselt_pred)
mape_nusselt = mean_absolute_percentage_error(y_test['nusselt'], nusselt_pred)

# Calcular R^2 y MSE para nusselt_gp
r2_nusselt_gp = r2_score(y_test['nusselt'], nusselt_gp_pred)
mse_nusselt_gp = mean_squared_error(y_test['nusselt'], nusselt_gp_pred)

# Calcular MAE y MAPE para nusselt_gp
mae_nusselt_gp = mean_absolute_error(y_test['nusselt'], nusselt_gp_pred)
mape_nusselt_gp = mean_absolute_percentage_error(y_test['nusselt'], nusselt_gp_pred)

# Mostrar Métricas
print(f"R^2 nusselt original: {r2_nusselt}, R^2 nusselt GPLearn: {r2_nusselt_gp} ")
print(f"MSE nusselt original: {mse_nusselt}, MSE nusselt GPLearn: {mse_nusselt_gp} ")
print(f"MAE nusselt original: {mae_nusselt}, MAE nusselt GPLearn: {mae_nusselt_gp} ")
print(f"MAPE nusselt original: {mape_nusselt*100} %, MAPE nusselt GPLearn: {mape_nusselt_gp*100} % ")

R^2 nusselt original: 0.9509992536047758, R^2 nusselt GPLearn: 0.9812929358818617 
MSE nusselt original: 23.767192225174124, MSE nusselt GPLearn: 9.0736248235554 
MAE nusselt original: 4.174721418276441, MAE nusselt GPLearn: 1.8080640092076876 
MAPE nusselt original: 7.197154998836116 %, MAPE nusselt GPLearn: 3.0491436404132743 % 


In [12]:
# Ecuacion 6

def nusselt(S, Re, Pr):
    return 0.5 * S **(-0.2) * Re **(0.64) * Pr

def nusselt_gp(S, Re, Pr):
    return ((Re ** 0.5267087) + 0.708955) - (S / 0.08209341)

# Calcular nusselt_pred y nusselt_gp_pred en vectores separados
nusselt_pred = X_test.apply(lambda row: nusselt(row['K'], row['Re'], row['prandtl']), axis=1).values
nusselt_gp_pred = X_test.apply(lambda row: nusselt_gp(row['K'], row['Re'], row['prandtl']), axis=1).values

# Calcular R^2 y MSE para nusselt
r2_nusselt = r2_score(y_test['nusselt'], nusselt_pred)
mse_nusselt = mean_squared_error(y_test['nusselt'], nusselt_pred)

# Calcular MAE y MAPE para nusselt
mae_nusselt = mean_absolute_error(y_test['nusselt'], nusselt_pred)
mape_nusselt = mean_absolute_percentage_error(y_test['nusselt'], nusselt_pred)

# Calcular R^2 y MSE para nusselt_gp
r2_nusselt_gp = r2_score(y_test['nusselt'], nusselt_gp_pred)
mse_nusselt_gp = mean_squared_error(y_test['nusselt'], nusselt_gp_pred)

# Calcular MAE y MAPE para nusselt_gp
mae_nusselt_gp = mean_absolute_error(y_test['nusselt'], nusselt_gp_pred)
mape_nusselt_gp = mean_absolute_percentage_error(y_test['nusselt'], nusselt_gp_pred)

# Mostrar Métricas
print(f"R^2 nusselt original: {r2_nusselt}, R^2 nusselt GPLearn: {r2_nusselt_gp} ")
print(f"MSE nusselt original: {mse_nusselt}, MSE nusselt GPLearn: {mse_nusselt_gp} ")
print(f"MAE nusselt original: {mae_nusselt}, MAE nusselt GPLearn: {mae_nusselt_gp} ")
print(f"MAPE nusselt original: {mape_nusselt*100} %, MAPE nusselt GPLearn: {mape_nusselt_gp*100} % ")

R^2 nusselt original: 0.9509992536047758, R^2 nusselt GPLearn: 0.9816889106293251 
MSE nusselt original: 23.767192225174124, MSE nusselt GPLearn: 8.881562281010277 
MAE nusselt original: 4.174721418276441, MAE nusselt GPLearn: 1.776796774305004 
MAPE nusselt original: 7.197154998836116 %, MAPE nusselt GPLearn: 2.9910626295292992 % 


In [13]:
# Ecuacion 6 con Pr

def nusselt(S, Re, Pr):
    return 0.5 * S **(-0.2) * Re **(0.64) * Pr

def nusselt_gp(S, Re, Pr):
    return ((Re ** 0.5267087) + Pr) - (S / 0.08209341)

# Calcular nusselt_pred y nusselt_gp_pred en vectores separados
nusselt_pred = X_test.apply(lambda row: nusselt(row['K'], row['Re'], row['prandtl']), axis=1).values
nusselt_gp_pred = X_test.apply(lambda row: nusselt_gp(row['K'], row['Re'], row['prandtl']), axis=1).values

# Calcular R^2 y MSE para nusselt
r2_nusselt = r2_score(y_test['nusselt'], nusselt_pred)
mse_nusselt = mean_squared_error(y_test['nusselt'], nusselt_pred)

# Calcular MAE y MAPE para nusselt
mae_nusselt = mean_absolute_error(y_test['nusselt'], nusselt_pred)
mape_nusselt = mean_absolute_percentage_error(y_test['nusselt'], nusselt_pred)

# Calcular R^2 y MSE para nusselt_gp
r2_nusselt_gp = r2_score(y_test['nusselt'], nusselt_gp_pred)
mse_nusselt_gp = mean_squared_error(y_test['nusselt'], nusselt_gp_pred)

# Calcular MAE y MAPE para nusselt_gp
mae_nusselt_gp = mean_absolute_error(y_test['nusselt'], nusselt_gp_pred)
mape_nusselt_gp = mean_absolute_percentage_error(y_test['nusselt'], nusselt_gp_pred)

# Mostrar Métricas
print(f"R^2 nusselt original: {r2_nusselt}, R^2 nusselt GPLearn: {r2_nusselt_gp} ")
print(f"MSE nusselt original: {mse_nusselt}, MSE nusselt GPLearn: {mse_nusselt_gp} ")
print(f"MAE nusselt original: {mae_nusselt}, MAE nusselt GPLearn: {mae_nusselt_gp} ")
print(f"MAPE nusselt original: {mape_nusselt*100} %, MAPE nusselt GPLearn: {mape_nusselt_gp*100} % ")

R^2 nusselt original: 0.9509992536047758, R^2 nusselt GPLearn: 0.981686661671335 
MSE nusselt original: 23.767192225174124, MSE nusselt GPLearn: 8.88265310963614 
MAE nusselt original: 4.174721418276441, MAE nusselt GPLearn: 1.7768580928853435 
MAPE nusselt original: 7.197154998836116 %, MAPE nusselt GPLearn: 2.991207333588572 % 


In [11]:
# Ecuacion 8

def nusselt(S, Re, Pr):
    return 0.5 * S **(-0.2) * Re **(0.64) * Pr

def nusselt_gp(S, Re, Pr):
    return (((Re / ((S +1.5562664) / 0.30350396)) ** 0.6763588) - 0.09196712) + 2.6371925

# Calcular nusselt_pred y nusselt_gp_pred en vectores separados
nusselt_pred = X_test.apply(lambda row: nusselt(row['K'], row['Re'], row['prandtl']), axis=1).values
nusselt_gp_pred = X_test.apply(lambda row: nusselt_gp(row['K'], row['Re'], row['prandtl']), axis=1).values

# Calcular R^2 y MSE para nusselt
r2_nusselt = r2_score(y_test['nusselt'], nusselt_pred)
mse_nusselt = mean_squared_error(y_test['nusselt'], nusselt_pred)

# Calcular MAE y MAPE para nusselt
mae_nusselt = mean_absolute_error(y_test['nusselt'], nusselt_pred)
mape_nusselt = mean_absolute_percentage_error(y_test['nusselt'], nusselt_pred)

# Calcular R^2 y MSE para nusselt_gp
r2_nusselt_gp = r2_score(y_test['nusselt'], nusselt_gp_pred)
mse_nusselt_gp = mean_squared_error(y_test['nusselt'], nusselt_gp_pred)

# Calcular MAE y MAPE para nusselt_gp
mae_nusselt_gp = mean_absolute_error(y_test['nusselt'], nusselt_gp_pred)
mape_nusselt_gp = mean_absolute_percentage_error(y_test['nusselt'], nusselt_gp_pred)

# Mostrar Métricas
print(f"R^2 nusselt original: {r2_nusselt}, R^2 nusselt GPLearn: {r2_nusselt_gp} ")
print(f"MSE nusselt original: {mse_nusselt}, MSE nusselt GPLearn: {mse_nusselt_gp} ")
print(f"MAE nusselt original: {mae_nusselt}, MAE nusselt GPLearn: {mae_nusselt_gp} ")
print(f"MAPE nusselt original: {mape_nusselt*100} %, MAPE nusselt GPLearn: {mape_nusselt_gp*100} % ")

R^2 nusselt original: 0.9509992536047758, R^2 nusselt GPLearn: 0.9889588213639973 
MSE nusselt original: 23.767192225174124, MSE nusselt GPLearn: 5.35538403676111 
MAE nusselt original: 4.174721418276441, MAE nusselt GPLearn: 1.560743176492877 
MAPE nusselt original: 7.197154998836116 %, MAPE nusselt GPLearn: 2.6526222373546364 % 


### Experimento, incorporar exponentes de RDlS

In [6]:
model = PySRRegressor(
    niterations=30,
    binary_operators=["+","-","*" ,"/", "^"],
    unary_operators=["pot1(x)= x >= 0 ?  convert(typeof(x),x^(-0.2)) : convert(typeof(x), NaN)",
                    "pot2(x)=x >= 0 ?  convert(typeof(x),x^(0.64)) : convert(typeof(x), NaN)"],
    extra_sympy_mappings={"pot1": lambda x:  x**(-0.2) , 
                        "pot2": lambda x: x**(0.64)},
    population_size=200,
    verbosity=1,
    maxsize=12,
    parsimony=1e-3,
    progress=True
)

model.fit(X_train, y_train)

d:\env\lib\site-packages\pysr\sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
d:\env\lib\site-packages\pysr\sr.py:2029: UserWarning: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (https://astroautomata.com/PySR/options/#batching). You should also reconsider if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form with symbolic regression. More datapoints will lower the search speed.
  warnings.warn(


Compiling Julia backend...


d:\env\lib\site-packages\pysr\sr.py:73: UserWarning: You are using the `^` operator, but have not set up `constraints` for it. This may lead to overly complex expressions. One typical constraint is to use `constraints={..., '^': (-1, 1)}`, which will allow arbitrary-complexity base (-1) but only powers such as a constant or variable (1). For more tips, please see https://astroautomata.com/PySR/tuning/
  warnings.warn(
[ Info: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (`options.batching`), and also if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form.
[ Info: Started!



Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 450 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 5.480e+01
Head worker occupation: 0.0%
Progress: 1 / 450 total iterations (0.222%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
3           3.604e+03  5.314e+00  y = K + 1.3803
4           2.107e+03  5.367e-01  y = pot2(Re) * 0.57839
5           1.286e+03  4.935e-01  y = pot2(pot2(Re)) - prandtl
6           3.103e+02  1.422e+00  y = pot2(Re ^ (prandtl + 0.13061))
8           2.100e+01  1.346e+00  y = -9.6244 + ((pot1(Re) ^ -

JuliaError: SystemError: close: Invalid argument
Stacktrace:
  [1] systemerror(p::String, errno::Int32; extrainfo::Nothing)
    @ Base .\error.jl:176
  [2] systemerror
    @ .\error.jl:175 [inlined]
  [3] close(s::IOStream)
    @ Base .\iostream.jl:63
  [4] open(::SymbolicRegression.SearchUtilsModule.var"#38#41"{String}, ::String, ::Vararg{String}; kwargs::@Kwargs{})
    @ Base .\io.jl:398
  [5] open
    @ .\io.jl:393 [inlined]
  [6] save_to_file(dominating::Vector{PopMember{Float32, Float32, Node{Float32}}}, nout::Int64, j::Int64, dataset::Dataset{Float32, Float32, Matrix{Float32}, Vector{Float32}, Nothing, @NamedTuple{}, Nothing, Nothing, Nothing, Nothing}, options::Options{SymbolicRegression.CoreModule.OptionsStructModule.ComplexityMapping{Int64, Int64}, DynamicExpressions.OperatorEnumModule.OperatorEnum, Node, false, false, nothing, StatsBase.Weights{Float64, Float64, Vector{Float64}}})
    @ SymbolicRegression.SearchUtilsModule C:\Users\lucas\.julia\packages\SymbolicRegression\9q4